In [1]:
import sys
sys.path.append('../..')
from src.core.GameInstance import apply_react, reaction, GameInstance
from src.core.GameState import Location, Aura, GameState, Box
from src.game import activate, Game, fake_callback
b1 = Box(['Fischl','Ayaka','Sucrose'],[])
b2 = Box(['Diluc','Kaeya','Sucrose'],[])
state = GameState(b1,b2)
g = Game(state)
g.initiate(0, 0, fake_callback)
from agent.alphabeta import Searcher
from src.core.Event import Event, Death, Switch
from agent.MCTS import *

In [2]:
net = CatNet()

In [3]:
net.to('cuda')

CatNet(
  (hid): Sequential(
    (0): Linear(in_features=190, out_features=380, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(380, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=380, out_features=190, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(190, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=190, out_features=50, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (policy_head): Sequential(
    (0): Linear(in_features=50, out_features=6, bias=True)
    (1): LogSoftmax(dim=-1)
  )
  (value_head): Sequential(
    (0): Linear(in_features=50, out_features=1, bias=True)
    (1): Tanh()
  )
)

In [4]:
import torch
checkpoint = torch.load('./catnet.pt')
net.load_state_dict(checkpoint['model'])
mcts = MCTS(net,'../../agent/ids.json')
coach = Trainer(mcts, '.')
randomplayer = RandomPlayer()

In [5]:
#coach.train(20, 10, sims = 20)
#coach.test(Searcher(), 20, time_per_step = 0.05, verbose = True)

In [ ]:
def input_callback(g, event, *args):
    #print("yes i am here")
    if isinstance(event, Death) and g.history['phase'] == 'deathswitch':
        active = g.getactive(event.player_id)
        direct = int(input('your switch(1 for next, -1 for previous):'))
        return Switch(g.nexteid(), event.eid, event.player_id, active, direct)
    else:
        return Event(-1,-1,-1)
player = int(input('your player_id is:'))
Master = mcts
def callback(g, event, *args):
    if event.player_id == player:
        return input_callback(g,event,*args)
    else:
        return Master.callback(g,event,*args)
while not g.terminated():
    #player = 1
    print("mover = ", g.mover)
    if g.mover != player:
        #m, s, d = Master.search(5, g)
        m = Master.search(g,5)
        #m, s, d = Rival.search(5, g)
        #m = g.getIns(1, 'switch next')
        print("move = ", m)
        g.proceed(m,False, callback)
        #nodes = Master.nodes
        print("pi = ", Master.pi)
        #print("depth = {}, total nodes = {}, score = {}".format(d,nodes,s))
    elif g.mover == player:
        m = input("your move:")
        m = g.getIns(player, m)
        #m, s, d = Rival.search(5, g)
        g.proceed(m,False, callback)
        #print("move = ",m)
        #nodes = Rival.nodes
    else:
        print("mover = ", g.mover)
        break
    #print("player {} makes a move {}".format(player, m))
    print("hp distribution: ")
    print("p1 = ",[c.hp for c in g.g.p1.char],'\t\t','p2 = ', [c.hp for c in g.g.p2.char] )
    print("p1 active = ",g.g.p1.history['active'], ' p2 active = ', g.g.p2.history['active'])
    print("p1 end = ",g.g.p1.history['endround'], 'p2 end = ',g.g.p2.history['endround'])
    print("p1 aura = ", g.g.p1.getAura(), ' p2 aura = ', g.g.p2.getAura())
    print()

your player_id is: 1


mover =  1


your move: skill


hp distribution: 
p1 =  [10, 10, 10] 		 p2 =  [9, 10, 10]
p1 active =  0  p2 active =  0
p1 end =  False p2 end =  False
p1 aura =  [<Aura.empty: 0>, <Aura.empty: 0>, <Aura.empty: 0>]  p2 aura =  [<Aura.electro: 3>, <Aura.empty: 0>, <Aura.empty: 0>]

mover =  2
move =  UseKit(player_id=2, dice_pattern=DicePattern(pyro=3, cryo=0, hydro=0, electro=0, dendro=0, anemo=0, geo=0, black=0, white=0, omni=0), real_dice_pattern=DicePattern(pyro=3, cryo=0, hydro=0, electro=0, dendro=0, anemo=0, geo=0, black=0, white=0, omni=0), dice_instance=DiceInstance(omni=3, pyro=0, cryo=0, hydro=0, electro=0, dendro=0, anemo=0, geo=0), kit='skill')
pi =  tensor([0., 1., 0., 0., 0., 0.])
hp distribution: 
p1 =  [7, 10, 10] 		 p2 =  [9, 10, 10]
p1 active =  0  p2 active =  0
p1 end =  False p2 end =  False
p1 aura =  [<Aura.pyro: 1>, <Aura.empty: 0>, <Aura.empty: 0>]  p2 aura =  [<Aura.electro: 3>, <Aura.empty: 0>, <Aura.empty: 0>]

mover =  1
